In [30]:
!pip install geopy scikit-learn matplotlib gmplot

In [36]:
import time
import math
import numpy as np
from geopy.geocoders import Nominatim
from geopy import distance
from sklearn.manifold import MDS
import matplotlib.pyplot as plt
import gmplot

def fetch_coordinates(names: list[str]) -> list[tuple[float, float, float]]:
    """
    取得每個站點的 (緯度, 經度, 海拔)。
    因為 Nominatim 無法回傳海拔，這裡暫時把 altitude 都設為 0.0 (公里)。
    如果之後你有真正的海拔（m 單位），請先轉成 km 填在第三個欄位。
    """
    geolocator = Nominatim(user_agent="mds-3d-demo")
    coords = []
    for name in names:
        location = geolocator.geocode(name)
        if location is None:
            raise RuntimeError(f"Failed to geocode \"{name}\"")
        lat = location.latitude
        lon = location.longitude
        alt = 0.0  # 單位：公里。如果你有海拔 (m)，請除以 1000 再放進來。
        coords.append((lat, lon, alt))
        time.sleep(1)  # 避免被 API rate limit
    return coords

def compute_3d_distance_matrix(coords: list[tuple[float, float, float]]) -> np.ndarray:
    """
    給定 coords = [(lat1, lon1, alt1), (lat2, lon2, alt2), ...]
    先用 geopy.distance.distance() 計算平面距離 (lat/lon) → 單位 km，
    再加上 alt 差做歐氏距離，回傳一個 n×n 的距離矩陣 (km)。
    """
    n = len(coords)
    mat = np.zeros((n, n), dtype=float)
    for i in range(n):
        for j in range(i+1, n):
            lat1, lon1, alt1 = coords[i]
            lat2, lon2, alt2 = coords[j]

            # 1. 平面距離 (geodesic distance in km)
            flat_dist_km = distance.distance((lat1, lon1), (lat2, lon2)).km

            # 2. 高度差 (km)
            delta_alt = alt2 - alt1

            # 3. 歐氏距離 (3D distance)
            euclid_dist = math.sqrt(flat_dist_km**2 + delta_alt**2)

            mat[i, j] = euclid_dist
            mat[j, i] = euclid_dist
    return mat

def run_mds(dist_matrix: np.ndarray) -> np.ndarray:
    """
    對 precomputed 的距離矩陣做 classical MDS，得到 2D 坐標。
    """
    mds = MDS(n_components=2, dissimilarity="precomputed", random_state=0)
    embedding = mds.fit_transform(dist_matrix)
    return embedding

def plot_embedding(points: np.ndarray, labels: list[str], outfile: str) -> None:
    """
    繪製 2D embedding 並存檔。所有文字都用英文。
    """
    plt.figure(figsize=(8, 6))
    plt.scatter(points[:, 0], points[:, 1], color="steelblue", s=60)
    for (x, y), label in zip(points, labels):
        plt.annotate(label, (x, y), textcoords="offset points", xytext=(5,5), fontsize=10)
    plt.title("MDS of Taiwanese Train Stations (3D Distance)", fontsize=14)
    plt.xlabel("Dimension 1")
    plt.ylabel("Dimension 2")
    plt.tight_layout()
    plt.savefig(outfile, dpi=200)
    plt.close()
    print(f"Saved embedding plot as: {outfile}")

def generate_google_map(
    coords: list[tuple[float, float, float]], names: list[str], outfile: str
) -> None:
    """
    產生一個帶有站名標記的 Google Map HTML（標題皆為英文）。
    因為 gmplot 只能接受 (lat, lon)，這裡忽略 alt。
    """
    latitudes, longitudes, _ = zip(*coords)
    center_lat = sum(latitudes) / len(latitudes)
    center_lon = sum(longitudes) / len(longitudes)

    # 如果有 API Key，可以這樣傳入：apikey="YOUR_API_KEY"
    gmap = gmplot.GoogleMapPlotter(center_lat, center_lon, zoom=7)

    for lat, lon, name in zip(latitudes, longitudes, names):
        gmap.marker(lat, lon, title=name)
    gmap.draw(outfile)
    print(f"Saved Google Map HTML as: {outfile}")

def main() -> None:
    # 這裡 station_names 用英文名稱並加上 ", Taiwan"
    station_names = [
        "Taipei Main Station, Taiwan",
        "Hsinchu Station, Taiwan",
        "Taichung Station, Taiwan",
        "Douliu Station, Taiwan",
        "Kaohsiung Station, Taiwan",
        "Yuli Station, Hualien, Taiwan",
        "Zhiben Station, Taitung ,Taiwan",
    ]


    coords = fetch_coordinates(station_names)
    dist_matrix = compute_3d_distance_matrix(coords)
    embedding = run_mds(dist_matrix)

    # 再產生地圖檔 (不受旋轉影響，地圖還是 lat/lon)
    generate_google_map(coords, station_names, "stations_map.html")
    print("All results saved: mds_plot.png, stations_map.html")

    np.set_printoptions(precision=3, suppress=True)
    print("\nDistance matrix (3D, unit: km):")
    print(dist_matrix)

    print("\nAll results saved: mds_plot.png, stations_map.html")

if __name__ == "__main__":
    main()


Saved Google Map HTML as: stations_map.html
All results saved: mds_plot.png, stations_map.html

Distance matrix (3D, unit: km):
[[  0.     61.481 131.321 178.039 293.876 191.227 263.038]
 [ 61.481   0.     79.047 128.36  248.807 166.427 231.792]
 [131.321  79.047   0.     49.381 170.276 109.616 162.58 ]
 [178.039 128.36   49.381   0.    121.07   89.238 123.016]
 [293.876 248.807 170.276 121.07    0.    128.69   78.344]
 [191.227 166.427 109.616  89.238 128.69    0.     73.464]
 [263.038 231.792 162.58  123.016  78.344  73.464   0.   ]]

All results saved: mds_plot.png, stations_map.html
